Since we are working with ex-vivo samples, they have some systematic differences from MRI which we try to recover here. First they often drift apart at the hippocampal sulcus, making it much wider. secondly, they are often dehydrated making the grey matter labels disproportionately thinner. Third, they often don't have separate labels for cysts, so we introduce some here scattered throughout the SRLM.

In [1]:
import ants
import nibabel as nib
import numpy as np
import os
from scipy.ndimage import binary_dilation, binary_erosion
import random

In [2]:
lbls = !ls labelmaps/*.nii.gz
lbls

['labelmaps/img00.nii.gz',
 'labelmaps/img01.nii.gz',
 'labelmaps/img02.nii.gz',
 'labelmaps/img03.nii.gz',
 'labelmaps/img04.nii.gz',
 'labelmaps/img05.nii.gz',
 'labelmaps/img06.nii.gz',
 'labelmaps/img07.nii.gz',
 'labelmaps/img08.nii.gz',
 'labelmaps/img09.nii.gz',
 'labelmaps/img10.nii.gz',
 'labelmaps/img11.nii.gz',
 'labelmaps/img12.nii.gz',
 'labelmaps/img13.nii.gz',
 'labelmaps/img14.nii.gz',
 'labelmaps/img15.nii.gz',
 'labelmaps/img16.nii.gz',
 'labelmaps/img17.nii.gz',
 'labelmaps/img18.nii.gz',
 'labelmaps/img19.nii.gz',
 'labelmaps/img20.nii.gz',
 'labelmaps/img21.nii.gz',
 'labelmaps/img22.nii.gz',
 'labelmaps/img23.nii.gz',
 'labelmaps/img24.nii.gz',
 'labelmaps/img25.nii.gz',
 'labelmaps/img26.nii.gz',
 'labelmaps/img27.nii.gz',
 'labelmaps/img28.nii.gz',
 'labelmaps/img29.nii.gz',
 'labelmaps/img30.nii.gz',
 'labelmaps/img31.nii.gz',
 'labelmaps/img32.nii.gz',
 'labelmaps/img33.nii.gz',
 'labelmaps/img34.nii.gz',
 'labelmaps/img35.nii.gz',
 'labelmaps/img36.nii.gz',
 

In [3]:
def fast_morph(image, labels, radius=1, preserve=[]):
    '''
    Note that if radius is negative, the label is actually being thinned. Since the registration is smooth, the actual deformation is smaller than the radius
    '''
    img = image.get_fdata().astype(int)
    mask = np.isin(img, labels).astype(float)
    if radius > 0:
        dilated_mask = binary_dilation(mask, iterations=np.abs(radius)).astype(float)
    else:
        dilated_mask = binary_erosion(mask, iterations=np.abs(radius)).astype(float)
    dilated_mask[np.isin(img, preserve)] = 0

    dilated_mask = ants.from_numpy(dilated_mask)
    mask = ants.from_numpy(mask)
    img = ants.from_numpy(img)

    outprefix = "/dev/shm/transform"
    reg = ants.registration(
        dilated_mask, mask, type_of_transform='SyNOnly',
        reg_iterations=[np.abs(radius)*4,0],syn_metric='demons',grad_step=0.5,
        outprefix=outprefix,
    )
    img_deformed = ants.apply_transforms(
        img, img, reg['fwdtransforms'], interpolator='genericLabel'
    )
    dilated_mask = img_deformed.numpy()
    for f in reg['fwdtransforms']:
        os.remove(f)

    return nib.Nifti1Image(dilated_mask, affine=image.affine, header=image.header)

def add_cysts(image, cyst_label, labels_in, ncysts, radius=2):
    img = image.get_fdata()

    possible_seeds = np.argwhere(np.isin(img, labels_in))
    seed_idxs = np.random.choice(len(possible_seeds), ncysts, replace=False)
    seeds = possible_seeds[seed_idxs]
    cysts = np.zeros_like(img)
    for seed in seeds:
        x,y,z = seed
        cysts[x,y,z] = 1
        
    # dilate all cysts
    cysts = binary_dilation(cysts, iterations=np.abs(radius))
    # remove cysts outside of labels_in
    cysts[~np.isin(img, labels_in)] = 0

    img[cysts==1] = cyst_label

    return nib.Nifti1Image(img, affine=image.affine, header=image.header)

In [4]:
# make a copy with a thinned srlm
i = len(lbls)
for lbl in lbls:
    image = nib.load(lbl)
    image = fast_morph(image, [7], radius=-1)
    nib.save(image, f"labelmaps/img{i:02d}.nii.gz")
    i+=1

In [5]:
# make a copy with a thicker hipp 
for lbl in lbls:
    image = nib.load(lbl)
    image = fast_morph(image, [1,2,3,4], radius=2)
    nib.save(image, f"labelmaps/img{i:02d}.nii.gz")
    i+=1

In [7]:
# for all of the above cases, also randomly add some cysts [8] only in srlm [7]
lbls = !ls labelmaps/*.nii.gz
for lbl in lbls:
    for ncysts in [5,10,15]:
        for cysts_rad in [2,4]:
            image = nib.load(lbl)
            image = add_cysts(image, [8], [7], ncysts, radius=cysts_rad)
            nib.save(image, f"labelmaps/img{i:02d}.nii.gz")
            i+=1